In [1]:
# libraries nneded for nlp
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()
## it generally use lancaster stemming algo in order to get the stem from words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install tflearn

In [3]:
#libraries needed for Tensorflow processing
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import json ## to read json file as intent is in json format
# intent file we will train model

In [4]:
from google.colab import files
files.upload()

Saving intents.json to intents (4).json


{'intents (4).json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where 

In [5]:
# import our chatbot intents file
with open('intents.json') as json_data:
  intents= json.load(json_data)

In [6]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [7]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

sentence = "Hello there! How are you doing today?"
tokens = tokenizer.tokenize(sentence)

print(tokens)


['Hello', 'there', 'How', 'are', 'you', 'doing', 'today']


In [8]:
import nltk
nltk.download('punkt_tab')

words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
#Perform stemming and lowering each word as well as remove duplicates using set keyword
words= [stemmer.stem(w.lower()) for w in words if w not in ignore]
words= sorted(list(set(words)))

# remove duplicate classes
classes= sorted(list(set(classes)))

In [10]:
# create training data
training =[]
output=[]
#create an empty array for output
output_empty=[0]*len(classes)

##create training set, bag of words for each sentence
for doc in documents:
  bag=[]
  #list of tokenized words for the pattern
  pattern_words= doc[0]
  #stem each word
  pattern_words= [stemmer.stem(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  #output is '1' for current tag and '0' for rest of the other tags
  output_row= list(output_empty)
  output_row[classes.index(doc[1])]=1

  training.append([bag,output_row])

## shuffling the features and turning it into np.array
random.shuffle(training)

train_x = []
train_y = []

for x, y in training:
    train_x.append(x)
    train_y.append(y)


In [11]:
import numpy as np

train_x = np.array(train_x, dtype='float32')
train_y = np.array(train_y, dtype='float32')


In [12]:
import numpy as np

train_x = np.array(train_x).astype(np.float32)
train_y = np.array(train_y).astype(np.float32)


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Clear old graph — especially useful in Colab
tf.keras.backend.clear_session()

# Ensure train_x and train_y are clean float32 numpy arrays
train_x = np.array(train_x).astype(np.float32)
train_y = np.array(train_y).astype(np.float32)

# Define the model using Input() to avoid warnings & shape issues
model = Sequential([
    Input(shape=(train_x.shape[1],)),         # Input layer
    Dense(10, activation='relu'),             # Hidden layer 1
    Dense(10, activation='relu'),             # Hidden layer 2
    Dense(train_y.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 789 (3.08 KB)

 Trainable params: 789 (3.08 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(train_x, train_y, epochs=200, batch_size=8, verbose=1)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0341 - loss: 2.2213  
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0800 - loss: 2.1898 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0550 - loss: 2.1768     
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0595 - loss: 2.2072     
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0637 - loss: 2.1943     
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0470 - loss: 2.1837     
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0929 - loss: 2.1581 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1262 - loss: 2.1392 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1474 - loss: 2.1337
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1474 - loss: 2.1219 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1054 - loss: 2.1285 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accu

In [16]:
model.save('model.keras') # saves the architecture+weights+optimizer state

In [17]:
import pickle
data ={
    "words": words,
  "classes": classes,
  "train_x": train_x,
  "train_y": train_y
}
with open('model.pkl', 'wb') as f:
  pickle.dump(data, f)

In [23]:
from tensorflow.keras.models import load_model
#loading model
model = load_model('model.keras')
#load training data or metadata
with open('model.pkl', 'rb') as f:
  data = pickle.load(f)

words= data['words']
classes= data['classes']
train_x= data['train_x']
train_y= data['train_y']

In [24]:
with open('intents.json') as json_data:
  intents= json.load(json_data)

In [25]:
def clean_up_sentence(sentence):
  #tokenizing the pattern
  sentence_words = nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
  #tokenizing the pattern
  sentence_words = clean_up_sentence(sentence)
  #generating bag of words
  bag= [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i]=1
        if show_details:
          print("found in bag: %s" %w)

  return (np.array(bag))

In [26]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
  # generate probablities from the model
  input_data= np.array([bow(sentence,words)])
  results = model.predict(input_data)[0]
  #filter out predictions below a threshold
  results= [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  #sort by strength of probablity
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  #return tuple of intent and probablity
  return return_list

def response(sentence, userID='123', show_details=False):
  results = classify(sentence)
  # if we have a classfication then find the matching intent tag
  if results:
    #loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        #find a tag matching the first result
        if i['tag'] == results[0][0]:
          # a random response from the intent
          return print(random.choice(i['responses']))

      results.pop(0)

In [27]:
classify('What are you hours of operation?')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


[('hours', np.float32(0.97949535))]

In [28]:
response('What is your hours of operation?')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Our hours are 9am-9pm every day


In [29]:
response("hi how are you today")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Hi there, how can I help?


In [32]:
response("what is your favourite food")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
You can check various delicacies given in the food menu at www.mymenu.com


In [40]:
##Add some context to the conversation i.e. Contextualization for altering question
# and create a data structure to hold user context
context ={}

ERROR_THRESHOLD=0.25
def classify(sentence):
    # generate bag of words
    input_data = bow(sentence, words)

    # reshape to match model input: (1, 57)
    input_data = np.reshape(input_data, (1, len(input_data)))

    # generate probabilities from the model
    results = model.predict(input_data)[0]

    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]

    # sort by probability
    results.sort(key=lambda x: x[1], reverse=True)

    # prepare result list
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))

    return return_list




In [41]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)

    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:

                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details:
                            print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                       (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details:
                            print('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)


In [43]:
response('how are you')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Hello, thanks for visiting


In [44]:
response('i want to order')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Hi there, how can I help?


In [46]:
response('What is the spciality for today')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Our speciality for today is Chicken Tikka


In [47]:
response('Hi There!', show_details=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
context: 
tag: greeting
Hi there, how can I help?
